In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
%pip install jupyterlab_play_cell_button 

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
df = pd.read_csv('C:\Airline data\mia_data2.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Airline data\\mia_data2.csv'

In [3]:
df.keys()

Index(['Unnamed: 0.1', 'Unnamed: 0', 'YEAR', 'MONTH', 'DAY_OF_WEEK',
       'OP_CARRIER_FL_NUM', 'ORIGIN', 'DEST', 'CRS_DEP_TIME', 'DEP_TIME',
       'DEP_DELAY', 'DEP_DELAY_NEW', 'CRS_ARR_TIME', 'ARR_TIME', 'ARR_DELAY',
       'ARR_DELAY_NEW', 'CANCELLED', 'CANCELLATION_CODE', 'DIVERTED',
       'CRS_ELAPSED_TIME', 'ACTUAL_ELAPSED_TIME', 'CARRIER_DELAY',
       'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY',
       'OP_CARRIER', 'WHY_DELAY', 'DAY', 'DATE', 'DEPARTURE_TIME',
       'ARRIVAL_TIME', 'ALL_LATE_ARR', 'ALL_LATE_DEP', 'LATE_ARR', 'LATE_DEP'],
      dtype='object')

In [4]:
# I want to create dummy values based on the why delay column
dummy = pd.get_dummies(df['WHY_DELAY'])

In [5]:
dummy

,0,carrier,late aircraft,nas,security,weather
0,1,0,0,0,0,0
1,1,0,0,0,0,0
2,1,0,0,0,0,0
3,1,0,0,0,0,0
4,1,0,0,0,0,0
...,...,...,...,...,...,...
736647,1,0,0,0,0,0
736648,1,0,0,0,0,0
736649,1,0,0,0,0,0
736650,0,0,0,1,0,0


In [6]:
df = pd.concat([df, dummy], axis = 1)
df.keys()

Index(['Unnamed: 0.1', 'Unnamed: 0', 'YEAR', 'MONTH', 'DAY_OF_WEEK',
       'OP_CARRIER_FL_NUM', 'ORIGIN', 'DEST', 'CRS_DEP_TIME', 'DEP_TIME',
       'DEP_DELAY', 'DEP_DELAY_NEW', 'CRS_ARR_TIME', 'ARR_TIME', 'ARR_DELAY',
       'ARR_DELAY_NEW', 'CANCELLED', 'CANCELLATION_CODE', 'DIVERTED',
       'CRS_ELAPSED_TIME', 'ACTUAL_ELAPSED_TIME', 'CARRIER_DELAY',
       'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY',
       'OP_CARRIER', 'WHY_DELAY', 'DAY', 'DATE', 'DEPARTURE_TIME',
       'ARRIVAL_TIME', 'ALL_LATE_ARR', 'ALL_LATE_DEP', 'LATE_ARR', 'LATE_DEP',
       '0', 'carrier', 'late aircraft', 'nas', 'security', 'weather'],
      dtype='object')

In [7]:
df.drop(columns = ['Unnamed: 0.1', 'Unnamed: 0', 'OP_CARRIER_FL_NUM', 'CANCELLATION_CODE', 'DIVERTED',  'CARRIER_DELAY','WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY', 'YEAR', 'MONTH', 'DAY', 'WHY_DELAY']  , inplace = True)

In [8]:
# I need to create two data frames to work on one that is about departures from MIA and the 
#other one that deals with arrivals.

df_arr = df[df.DEST == 'MIA']
df_dep = df[df.ORIGIN == 'MIA']

In [9]:
df_arr

,DAY_OF_WEEK,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,DEP_DELAY_NEW,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,...,ALL_LATE_ARR,ALL_LATE_DEP,LATE_ARR,LATE_DEP,0,carrier,late aircraft,nas,security,weather
0,6.0,STT,MIA,1535,1519.0,-16.0,0.0,1840,1835.0,-5.0,...,NaN,NaN,NaN,NaN,1,0,0,0,0,0
1,7.0,STT,MIA,1535,1550.0,15.0,15.0,1840,1840.0,0.0,...,NaN,1535.0,NaN,NaN,1,0,0,0,0,0
2,1.0,STT,MIA,1535,1550.0,15.0,15.0,1840,1846.0,6.0,...,1840.0,1535.0,NaN,NaN,1,0,0,0,0,0
3,2.0,STT,MIA,1535,1533.0,-2.0,0.0,1840,1829.0,-11.0,...,NaN,NaN,NaN,NaN,1,0,0,0,0,0
4,3.0,STT,MIA,1535,1557.0,22.0,22.0,1840,1848.0,8.0,...,1840.0,1535.0,NaN,NaN,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
736642,4.0,JFK,MIA,1150,1142.0,-8.0,0.0,1451,1444.0,-7.0,...,NaN,NaN,NaN,NaN,1,0,0,0,0,0
736644,4.0,BOS,MIA,755,752.0,-3.0,0.0,1125,1110.0,-15.0,...,NaN,NaN,NaN,NaN,1,0,0,0,0,0
736645,4.0,DTW,MIA,830,827.0,-3.0,0.0,1123,1120.0,-3.0,...,NaN,NaN,NaN,NaN,1,0,0,0,0,0
736647,4.0,LGA,MIA,725,721.0,-4.0,0.0,1016,1009.0,-7.0,...,NaN,NaN,NaN,NaN,1,0,0,0,0,0


In [10]:
df_arr.keys()

Index(['DAY_OF_WEEK', 'ORIGIN', 'DEST', 'CRS_DEP_TIME', 'DEP_TIME',
       'DEP_DELAY', 'DEP_DELAY_NEW', 'CRS_ARR_TIME', 'ARR_TIME', 'ARR_DELAY',
       'ARR_DELAY_NEW', 'CANCELLED', 'CRS_ELAPSED_TIME', 'ACTUAL_ELAPSED_TIME',
       'OP_CARRIER', 'DATE', 'DEPARTURE_TIME', 'ARRIVAL_TIME', 'ALL_LATE_ARR',
       'ALL_LATE_DEP', 'LATE_ARR', 'LATE_DEP', '0', 'carrier', 'late aircraft',
       'nas', 'security', 'weather'],
      dtype='object')

In [11]:
y = df_arr['weather']

In [10]:
df_arr.drop(columns = ['weather', 'DEST', 'DEPARTURE_TIME', 'DEP_TIME', 'DEP_DELAY', 'carrier' , 'CANCELLED', 'late aircraft', 'nas', 'security', 'ARR_DELAY_NEW' ], inplace = True)

C:\Users\tucso\AppData\Local\Temp\ipykernel_10672\822718329.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_arr.drop(columns = ['weather', 'DEST', 'DEPARTURE_TIME', 'DEP_TIME', 'DEP_DELAY', 'carrier' , 'CANCELLED', 'late aircraft', 'nas', 'security', 'ARR_DELAY_NEW' ], inplace = True)


In [12]:
df_arr

,DAY_OF_WEEK,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,DEP_DELAY_NEW,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,...,ALL_LATE_ARR,ALL_LATE_DEP,LATE_ARR,LATE_DEP,0,carrier,late aircraft,nas,security,weather
0,6.0,STT,MIA,1535,1519.0,-16.0,0.0,1840,1835.0,-5.0,...,NaN,NaN,NaN,NaN,1,0,0,0,0,0
1,7.0,STT,MIA,1535,1550.0,15.0,15.0,1840,1840.0,0.0,...,NaN,1535.0,NaN,NaN,1,0,0,0,0,0
2,1.0,STT,MIA,1535,1550.0,15.0,15.0,1840,1846.0,6.0,...,1840.0,1535.0,NaN,NaN,1,0,0,0,0,0
3,2.0,STT,MIA,1535,1533.0,-2.0,0.0,1840,1829.0,-11.0,...,NaN,NaN,NaN,NaN,1,0,0,0,0,0
4,3.0,STT,MIA,1535,1557.0,22.0,22.0,1840,1848.0,8.0,...,1840.0,1535.0,NaN,NaN,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
736642,4.0,JFK,MIA,1150,1142.0,-8.0,0.0,1451,1444.0,-7.0,...,NaN,NaN,NaN,NaN,1,0,0,0,0,0
736644,4.0,BOS,MIA,755,752.0,-3.0,0.0,1125,1110.0,-15.0,...,NaN,NaN,NaN,NaN,1,0,0,0,0,0
736645,4.0,DTW,MIA,830,827.0,-3.0,0.0,1123,1120.0,-3.0,...,NaN,NaN,NaN,NaN,1,0,0,0,0,0
736647,4.0,LGA,MIA,725,721.0,-4.0,0.0,1016,1009.0,-7.0,...,NaN,NaN,NaN,NaN,1,0,0,0,0,0


In [13]:
df_arr.keys()

array([  nan,  625., 1230., 1925., 1420.,  600.,  545., 1620., 1105.,
       1415., 1440., 1955., 1810., 1340., 1750., 2000., 1720., 1845.,
        710., 1830.,  835., 1920., 1100., 1210., 1835., 1608., 1010.,
       1600., 1755.,  905., 1715., 1050., 1640.,  720., 1125., 1805.,
       1055., 1905., 1240., 1300., 1500., 1605., 1820.,  910., 1535.,
       2025.,  615., 1735., 1940., 2015., 1800., 1345., 1935., 1020.,
       1919., 1220., 1305., 1825., 1930.,  735., 1540.,  715., 1650.,
       1515., 1245., 1850.,  900., 1730.,  809., 1745., 1405.,  700.,
       2010.,  730., 1110., 1529., 2200., 1140., 1900., 1355.,  925.,
       2035., 2350.,  855., 1129., 1505., 1410., 1725., 2150., 1205.,
       1530.,  655., 1635., 1000., 1235.,  630.,  605., 1610., 2045.,
       1729.,  640.,  620., 1855., 1710., 1315., 1459., 1840., 1311.,
       1359., 1740.,  755., 1630., 1625., 1614.,  800., 2050., 1335.,
       1455., 1700., 1135., 1251., 1027., 1425., 1655.,  845., 1200.,
       1645., 1555.,

In [14]:
X_arr_train, X_arr_test, y_arr_train, y_arr_test = train_test_split(df_arr, y, test_size = .33, random_state = 42)

In [15]:
y_dep = df_dep['weather']

In [17]:
df_dep.drop(columns = ['weather'])

,DAY_OF_WEEK,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,DEP_DELAY_NEW,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,...,ARRIVAL_TIME,ALL_LATE_ARR,ALL_LATE_DEP,LATE_ARR,LATE_DEP,0,carrier,late aircraft,nas,security
44,6.0,MIA,IAH,1450,1446.0,-4.0,0.0,1640,1633.0,-7.0,...,16:40:00,NaN,NaN,NaN,NaN,1,0,0,0,0
45,7.0,MIA,IAH,1450,1447.0,-3.0,0.0,1640,1633.0,-7.0,...,16:40:00,NaN,NaN,NaN,NaN,1,0,0,0,0
46,1.0,MIA,IAH,1450,1447.0,-3.0,0.0,1640,1622.0,-18.0,...,16:40:00,NaN,NaN,NaN,NaN,1,0,0,0,0
47,2.0,MIA,IAH,1450,1456.0,6.0,6.0,1640,1639.0,-1.0,...,16:40:00,NaN,1450.0,NaN,NaN,1,0,0,0,0
48,3.0,MIA,IAH,1450,1553.0,63.0,63.0,1640,1732.0,52.0,...,16:40:00,1640.0,1450.0,NaN,NaN,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
736643,4.0,MIA,LGA,1730,1720.0,-10.0,0.0,2032,2006.0,-26.0,...,20:32:00,NaN,NaN,NaN,NaN,1,0,0,0,0
736646,4.0,MIA,JFK,1510,1503.0,-7.0,0.0,1814,1747.0,-27.0,...,18:14:00,NaN,NaN,NaN,NaN,1,0,0,0,0
736649,4.0,MIA,MSP,1346,1339.0,-7.0,0.0,1627,1608.0,-19.0,...,16:27:00,NaN,NaN,NaN,NaN,1,0,0,0,0
736650,4.0,MIA,ATL,806,842.0,36.0,36.0,955,1034.0,39.0,...,09:55:00,955.0,806.0,NaN,NaN,0,0,0,1,0


In [18]:
df_dep.keys()

Index(['DAY_OF_WEEK', 'ORIGIN', 'DEST', 'CRS_DEP_TIME', 'DEP_TIME',
       'DEP_DELAY', 'DEP_DELAY_NEW', 'CRS_ARR_TIME', 'ARR_TIME', 'ARR_DELAY',
       'ARR_DELAY_NEW', 'CANCELLED', 'CRS_ELAPSED_TIME', 'ACTUAL_ELAPSED_TIME',
       'OP_CARRIER', 'DATE', 'DEPARTURE_TIME', 'ARRIVAL_TIME', 'ALL_LATE_ARR',
       'ALL_LATE_DEP', 'LATE_ARR', 'LATE_DEP', '0', 'carrier', 'late aircraft',
       'nas', 'security', 'weather'],
      dtype='object')

In [19]:
df_dep.drop(columns = ['ORIGIN', 'CRS_ARR_TIME', 'ARR_TIME', 'ARR_DELAY', 'ARR_DELAY_NEW', 'CANCELLED', 'ARRIVAL_TIME', 'ALL_LATE_ARR', 'LATE_ARR','carrier', 'late aircraft', 'nas', 'security'], inplace = True)

C:\Users\tucso\AppData\Local\Temp\ipykernel_15356\456973482.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dep.drop(columns = ['ORIGIN', 'CRS_ARR_TIME', 'ARR_TIME', 'ARR_DELAY', 'ARR_DELAY_NEW', 'CANCELLED', 'ARRIVAL_TIME', 'ALL_LATE_ARR', 'LATE_ARR','carrier', 'late aircraft', 'nas', 'security'], inplace = True)


In [20]:
df_dep

,DAY_OF_WEEK,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,DEP_DELAY_NEW,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,OP_CARRIER,DATE,DEPARTURE_TIME,ALL_LATE_DEP,LATE_DEP,0,weather
44,6.0,IAH,1450,1446.0,-4.0,0.0,170.0,167.0,NaN,2010-05-01,14:50:00,NaN,NaN,1,0
45,7.0,IAH,1450,1447.0,-3.0,0.0,170.0,166.0,NaN,2010-05-02,14:50:00,NaN,NaN,1,0
46,1.0,IAH,1450,1447.0,-3.0,0.0,170.0,155.0,NaN,2010-05-03,14:50:00,NaN,NaN,1,0
47,2.0,IAH,1450,1456.0,6.0,6.0,170.0,163.0,NaN,2010-05-04,14:50:00,1450.0,NaN,1,0
48,3.0,IAH,1450,1553.0,63.0,63.0,170.0,159.0,NaN,2010-05-05,14:50:00,1450.0,NaN,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
736643,4.0,LGA,1730,1720.0,-10.0,0.0,182.0,166.0,NaN,2021-09-16,17:30:00,NaN,NaN,1,0
736646,4.0,JFK,1510,1503.0,-7.0,0.0,184.0,164.0,NaN,2021-09-16,15:10:00,NaN,NaN,1,0
736649,4.0,MSP,1346,1339.0,-7.0,0.0,221.0,209.0,NaN,2021-09-16,13:46:00,NaN,NaN,1,0
736650,4.0,ATL,806,842.0,36.0,36.0,109.0,112.0,NaN,2021-09-16,08:06:00,806.0,NaN,0,0


In [21]:
X_dep_train, X_dep_test, y_dep_train, y_dep_test = train_test_split(df_dep, y_dep, test_size = .33, random_state = 42)